# Colab setup

In [1]:
import sys
# if "google.colab" in str(get_ipython()):
! {sys.executable} -m pip install pytorch-lifestream
! {sys.executable} -m pip install umap-learn
! {sys.executable} -m pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.4/163.4 kB 4.0 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 7.2 MB/s eta 0:00:00
  Created wheel for pytorch-lifestream: filename=pytorch_lifestream-0.6.0-py3-none-any.whl size=274670 sha256=e5719ec4bc0de4b25bf07e3aa7dcf48ce632dbfcf07bb4dea6563ec460a6ec99
  Stored in directory: /root/.cache/pip/wheels/90/76/b4/0a944bc7c5a69201e4d757cc54886971117a2a581740e7f11d
Successfully built pytorch-lifestream
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.9 MB/s eta 0:00:00


# CoLES-demo-multimodal

**In this demo, we will try to show how Multimodal CoLES handles event data of different modalities.**

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import os

if not os.path.exists("lightning_logs/CoLES-demo-multimodal"):
    !mkdir -p lightning_logs/CoLES-demo-multimodal

if not os.path.exists("CatBoostClassifier"):
    !mkdir -p CatBoostClassifier

if not os.path.exists("model"):
    !mkdir -p model

# Libraries

In [5]:
! pip install pytorch-lifestream

In [6]:
from functools import partial
from datetime import timedelta

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import catboost
import umap

import torch
import pytorch_lightning as pl
from torch.utils.data.dataloader import DataLoader
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.model_selection import train_test_split

from ptls.nn import TrxEncoder
from ptls.nn.seq_encoder.rnn_encoder import RnnEncoder
from ptls.frames import PtlsDataModule
from ptls.frames.coles import CoLESModule
from ptls.frames.coles.split_strategy import SampleSlices
from ptls.frames.coles.multimodal_dataset import MultiModalDataset
from ptls.frames.coles.multimodal_dataset import MultiModalIterableDataset
from ptls.frames.coles.multimodal_dataset import MultiModalSortTimeSeqEncoderContainer
from ptls.frames.coles.multimodal_inference_dataset import MultiModalInferenceDataset
from ptls.frames.coles.multimodal_inference_dataset import MultiModalInferenceIterableDataset
from ptls.frames.inference_module import InferenceModuleMultimodal
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.data_load import IterableProcessingDataset
from ptls.data_load.utils import collate_feature_dict
from ptls.data_load.datasets import MemoryMapDataset
from ptls.preprocessing import PandasDataPreprocessor

# Working with data

## Data load

In [7]:
transactions = pd.read_csv("https://huggingface.co/datasets/dllllb/transactions-gender/resolve/main/transactions.csv.gz?download=true", compression="gzip")
targets = pd.read_csv("https://huggingface.co/datasets/dllllb/transactions-gender/resolve/main/gender_train.csv?download=true")

In [8]:
transactions = transactions.dropna().reset_index(drop=True)
transactions

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
0,39026145,208 14:18:12,5499,1010,-7254.31,453799
1,39026145,208 14:24:23,5499,1010,-1392.47,453306
2,39026145,209 15:09:16,5499,1010,-2852.31,453306
3,39026145,209 15:13:51,5499,1010,-853.45,058435
4,39026145,209 15:13:17,5499,1010,-1410.44,058435
...,...,...,...,...,...,...
4084146,61870738,453 16:03:02,5499,1010,-5176.84,10217113
4084147,61870738,454 10:54:60,5411,1010,-1652.77,022915
4084148,61870738,454 14:23:59,5499,1010,-4687.23,10217113
4084149,61870738,454 16:11:53,5541,1110,-4491.83,RU570124


Here
1. `customer_id` is the id of some user
2. `tr_datetime` is the time of the transaction
3. `mcc_code` is, in fact, the mcc code of the transaction
4. `tr_type` is the type of transaction (what was paid for)
5. `amount` is the amount of the transaction
6. `term-id` is the id of the terminal where the transaction was carried out

We will predict the gender of the user based on his transactions.

In [9]:
targets

,customer_id,gender
0,10928546,1
1,69348468,1
2,61009479,0
3,74045822,0
4,27979606,1
...,...,...
8395,90417572,0
8396,66837341,0
8397,10758984,1
8398,11376556,0


In [10]:
n_cutomers = len(pd.unique(transactions["customer_id"]))
n_labeling_cutomers = len(pd.unique(targets["customer_id"]))

print("n_cutomers:", n_cutomers)
print("n_labeling_cutomers:", n_labeling_cutomers)

n_cutomers: 14973
n_labeling_cutomers: 8400


In [11]:
list(transactions.columns)

['customer_id', 'tr_datetime', 'mcc_code', 'tr_type', 'amount', 'term_id']

In [12]:
sourceA = transactions[["customer_id", "tr_datetime", "mcc_code", "term_id"]]
sourceB = transactions[["customer_id", "tr_datetime", "tr_type", "amount"]]

In [13]:
sourceA

,customer_id,tr_datetime,mcc_code,term_id
0,39026145,208 14:18:12,5499,453799
1,39026145,208 14:24:23,5499,453306
2,39026145,209 15:09:16,5499,453306
3,39026145,209 15:13:51,5499,058435
4,39026145,209 15:13:17,5499,058435
...,...,...,...,...
4084146,61870738,453 16:03:02,5499,10217113
4084147,61870738,454 10:54:60,5411,022915
4084148,61870738,454 14:23:59,5499,10217113
4084149,61870738,454 16:11:53,5541,RU570124


In [14]:
sourceB

,customer_id,tr_datetime,tr_type,amount
0,39026145,208 14:18:12,1010,-7254.31
1,39026145,208 14:24:23,1010,-1392.47
2,39026145,209 15:09:16,1010,-2852.31
3,39026145,209 15:13:51,1010,-853.45
4,39026145,209 15:13:17,1010,-1410.44
...,...,...,...,...
4084146,61870738,453 16:03:02,1010,-5176.84
4084147,61870738,454 10:54:60,1010,-1652.77
4084148,61870738,454 14:23:59,1010,-4687.23
4084149,61870738,454 16:11:53,1110,-4491.83


In [15]:
sourceA_drop_indices = np.random.choice(sourceA.index, 130000, replace=False)
sourceB_drop_indices = np.random.choice(sourceB.index, 420000, replace=False)

sourceA = sourceA.drop(sourceA_drop_indices).reset_index(drop=True)
sourceB = sourceB.drop(sourceB_drop_indices).reset_index(drop=True)

In [16]:
sourceA

,customer_id,tr_datetime,mcc_code,term_id
0,39026145,208 14:18:12,5499,453799
1,39026145,208 14:24:23,5499,453306
2,39026145,209 15:09:16,5499,453306
3,39026145,209 15:13:51,5499,058435
4,39026145,209 15:13:17,5499,058435
...,...,...,...,...
3954146,61870738,453 16:03:02,5499,10217113
3954147,61870738,454 10:54:60,5411,022915
3954148,61870738,454 14:23:59,5499,10217113
3954149,61870738,454 16:11:53,5541,RU570124


In [17]:
sourceB

,customer_id,tr_datetime,tr_type,amount
0,39026145,208 14:18:12,1010,-7254.31
1,39026145,208 14:24:23,1010,-1392.47
2,39026145,209 15:09:16,1010,-2852.31
3,39026145,209 15:13:51,1010,-853.45
4,39026145,209 15:13:17,1010,-1410.44
...,...,...,...,...
3664146,61870738,452 19:33:04,1110,-4491.83
3664147,61870738,454 10:54:60,1010,-1652.77
3664148,61870738,454 14:23:59,1010,-4687.23
3664149,61870738,454 16:11:53,1110,-4491.83


## Preprocessing

In [18]:
mcc_code_in = len(np.unique((sourceA["mcc_code"])))
term_id_in = len(np.unique((sourceA["term_id"])))
tr_type_in = len(np.unique((sourceB["tr_type"])))

print("mcc_code_in:", mcc_code_in)
print("term_id_in:", term_id_in)
print("tr_type_in", tr_type_in)

mcc_code_in: 184
term_id_in: 435172
tr_type_in 73


In [19]:
def tr_datetime_preprocess(tr_datetime):
    days, hms = tr_datetime.split()
    hh, mm, ss = hms.split(":")

    seconds = timedelta(hours=int(hh), minutes=int(mm), seconds=int(ss))
    seconds = seconds.total_seconds()
    seconds += int(days) * 24 * 3600

    return int(seconds)

In [20]:
sourceA["tr_datetime"] = sourceA["tr_datetime"].apply(tr_datetime_preprocess)
sourceB["tr_datetime"] = sourceB["tr_datetime"].apply(tr_datetime_preprocess)

In [21]:
sourceA_preprocessor = PandasDataPreprocessor(
    col_id="customer_id",
    col_event_time="tr_datetime",
    event_time_transformation="none",
    cols_category=["mcc_code", "term_id"],
    return_records=False,
)

sourceB_preprocessor = PandasDataPreprocessor(
    col_id="customer_id",
    col_event_time="tr_datetime",
    event_time_transformation="none",
    cols_numerical=["tr_type", "amount"],
    return_records=False,
)

In [22]:
processed_sourceA = sourceA_preprocessor.fit_transform(sourceA)
processed_sourceB = sourceB_preprocessor.fit_transform(sourceB)

In [23]:
processed_sourceA.columns = [
    "sourceA_" + str(col) if str(col) != "customer_id" else str(col)
    for col in processed_sourceA.columns
]

In [24]:
processed_sourceB.columns = [
    "sourceB_" + str(col) if str(col) != "customer_id" else str(col)
    for col in processed_sourceB.columns
]

In [25]:
joined_data = processed_sourceA.merge(processed_sourceB, how="outer", on="customer_id")

In [26]:
joined_data = joined_data.applymap(lambda x: torch.tensor([]) if pd.isna(x) else x)

In [27]:
train_df, test_df = train_test_split(joined_data,
                                     test_size=0.1,
                                     random_state=42)
train_df, valid_df = train_test_split(train_df,
                                      test_size=0.1,
                                      random_state=42)

In [28]:
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [29]:
train_dict = train_df.to_dict("records")
valid_dict = valid_df.to_dict("records")
test_dict = test_df.to_dict("records")

In [30]:
source_features = {
    "sourceA": ["event_time", "mcc_code", "term_id"],
    "sourceB": ["event_time", "tr_type", "amount"]
}

In [31]:
splitter = SampleSlices(split_count=5, cnt_min=25, cnt_max=50)

In [32]:
train_multimodal_data = MultiModalIterableDataset(
    data = train_dict,
    splitter = splitter,
    source_features = source_features,
    col_id = "customer_id",
    col_time = "event_time",
    source_names = ("sourceA", "sourceB")
)

valid_multimodal_data = MultiModalIterableDataset(
    data = valid_dict,
    splitter = splitter,
    source_features = source_features,
    col_id = "customer_id",
    col_time = "event_time",
    source_names = ("sourceA", "sourceB")
)

In [33]:
train_loader = PtlsDataModule(
    train_data = train_multimodal_data,
    train_num_workers = 16,
    train_batch_size = 64,

    valid_data = valid_multimodal_data
)

In [34]:
sourceA_encoder_params = dict(
    embeddings_noise = 0.003,
    linear_projection_size = 64,
    embeddings = {
        "mcc_code": {"in": mcc_code_in, "out": 32},
        "term_id": {"in": term_id_in, "out": 32}
    },
)

sourceB_encoder_params = dict(
    embeddings_noise = 0.003,
    linear_projection_size = 64,
    embeddings = {
        "tr_type": {"in": tr_type_in, "out": 32},
    },
    numeric_values = {"amount": "identity"},
)

In [35]:
sourceA_encoder = TrxEncoder(**sourceA_encoder_params)
sourceB_encoder = TrxEncoder(**sourceB_encoder_params)

In [36]:
seq_encoder = MultiModalSortTimeSeqEncoderContainer(
    trx_encoders = {
        "sourceA": sourceA_encoder,
        "sourceB": sourceB_encoder,
    },

    input_size = 64,
    hidden_size = 256,
    seq_encoder_cls = RnnEncoder,
    type = "gru"
)

In [37]:
model = CoLESModule(
    seq_encoder = seq_encoder,
    optimizer_partial = partial(torch.optim.Adam, lr=0.004),
    lr_scheduler_partial = partial(torch.optim.lr_scheduler.StepLR, step_size=30, gamma=0.5)
)

In [38]:
# model.load_state_dict(torch.load("model/multimodal_coles.pt"))

In [39]:
logger = TensorBoardLogger("lightning_logs", name="CoLES-demo-multimodal")

pl_trainer = pl.Trainer(
    logger = logger,
    max_epochs = 1,
    accelerator = "gpu",
    devices = 1,
    enable_progress_bar = True
)

In [41]:
pl_trainer.fit(model, train_loader)

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [43]:
inf_test_data = MultiModalInferenceIterableDataset(
    data = test_dict,
    source_features = source_features,
    col_id = "customer_id",
    col_time = "event_time",
    source_names = ("sourceA", "sourceB")
)

In [44]:
inf_test_loader = DataLoader(
    dataset = inf_test_data,
    collate_fn = partial(inf_test_data.collate_fn, col_id="customer_id"),
    shuffle = False,
    num_workers = 0,
    batch_size = 8
)

In [45]:
inf_train_data = MultiModalInferenceIterableDataset(
    data = train_dict,
    source_features = source_features,
    col_id = "customer_id",
    col_time = "event_time",
    source_names = ("sourceA", "sourceB")
)

In [46]:
inf_train_loader = DataLoader(
    dataset = inf_train_data,
    collate_fn = partial(inf_train_data.collate_fn, col_id="customer_id"),
    shuffle = False,
    num_workers = 0,
    batch_size = 8
)

In [47]:
# inference_module = InferenceModuleMultimodal(
#     model = model,
#     pandas_output = True,
#     drop_seq_features = True,
#     model_out_name = "emb",
#     col_id = "customer_id"
# )

# inference_module.model.is_reduce_sequence = True

In [48]:
# inf_test_embeddings = pd.concat(
#     pl_trainer.predict(inference_module, inf_test_loader),
#     axis = 0
# )

In [49]:
# inf_test_embeddings

## Tunning hyperparams using RankMe metric

In [58]:
!git clone https://github.com/google-research/google-research.git

Cloning into 'google-research'...
remote: Enumerating objects: 93554, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 93554 (delta 8), reused 4 (delta 2), pack-reused 93525 (from 3)
Receiving objects: 100% (93554/93554), 1.08 GiB | 22.67 MiB/s, done.
Resolving deltas: 100% (58482/58482), done.
Updating files: 100% (22188/22188), done.


In [59]:
!ls google-research/graph_embedding/metrics

example.py  metrics.py	README.md


In [60]:
import sys
sys.path.append("google-research/graph_embedding/metrics")

In [61]:
from metrics import (rankme,
        coherence,
        pseudo_condition_number,
        alpha_req,
        stable_rank,
        ne_sum,
        self_clustering)

In [62]:
from itertools import product

In [63]:
!pip install git+https://github.com/simonzhang00/ripser-plusplus.git

  Cloning https://github.com/simonzhang00/ripser-plusplus.git to /tmp/pip-req-build-wstsbxko
  Running command git clone --filter=blob:none --quiet https://github.com/simonzhang00/ripser-plusplus.git /tmp/pip-req-build-wstsbxko
  Resolved https://github.com/simonzhang00/ripser-plusplus.git to commit 30243c0c752de26d7fdf6e41f08bf7b840ca4744
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
  Created wheel for ripserplusplus: filename=ripserplusplus-1.1.3-cp310-cp310-linux_x86_64.whl size=571117 sha256=582c0bc05e840f44d4ceab6b9e04e4fe9de0efca3a89b12729b42284d940b02d
  Stored in directory: /tmp/pip-ephem-wheel-cache-ra428g93/wheels/0e/ba/fa/b0e13a51d380910fa3550fe884e5899867be8161c9e7f4bc8e
Successfully built ripserplusplus


In [64]:
!ls 

CatBoostClassifier  google-research  lightning_logs  model


In [65]:
import ripserplusplus as rpp
def ripser_metric(embeddings):
    """Вычисление метрики на основе ripserplusplus."""
    start_time = time()  # Засекаем время
    
    if not isinstance(embeddings, np.ndarray):
        embeddings = np.array(embeddings)

    # Используем rpp.run с указанием режима обработки
    diagrams = rpp.run("--format point-cloud", embeddings)

    # Пример вычисления: суммируем длины всех "жизненных циклов" из диаграммы
    persistence_sum = sum([birth - death for birth, death in diagrams[0] if death > birth])

    elapsed_time = time() - start_time  # Вычисляем время
    # print(f"Computed ripser metric in {elapsed_time:.4f} seconds")

    return persistence_sum, elapsed_time

In [66]:
# batch_sizes = [64, 128]
# learning_rates = [0.001, 0.004]
# split_counts = [3, 5]
# cnt_min_values = [10, 25]
# cnt_max_values = [50, 100]

# # Генерация сетки гиперпараметров
# hyperparameter_grid = [
#     {
#         "batch_size": batch_size,
#         "learning_rate": lr,
#         "split_count": split_count,
#         "cnt_min": cnt_min,
#         "cnt_max": cnt_max,
#     }
#     for batch_size, lr, split_count, cnt_min, cnt_max in product(
#         batch_sizes, learning_rates, split_counts, cnt_min_values, cnt_max_values
#     )
# ]

In [67]:
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

In [68]:
checkpoints_path = "checkpoints"
os.makedirs(checkpoints_path, exist_ok=True)

In [69]:
def create_datasets(train_dict, valid_dict, params, source_features):
    splitter = SampleSlices(
        split_count=params["split_count"],
        cnt_min=params["cnt_min"],
        cnt_max=params["cnt_max"],
    )

    train_data = MultiModalIterableDataset(
        data=train_dict,
        splitter=splitter,
        source_features=source_features,
        col_id="customer_id",
        col_time="event_time",
        source_names=("sourceA", "sourceB"),
    )

    valid_data = MultiModalIterableDataset(
        data=valid_dict,
        splitter=splitter,
        source_features=source_features,
        col_id="customer_id",
        col_time="event_time",
        source_names=("sourceA", "sourceB"),
    )

    data_loader = PtlsDataModule(
        train_data=train_data,
        train_batch_size=params["batch_size"],
        train_num_workers=0,
        valid_data=valid_data,
    )

    return data_loader

In [70]:
from time import time
import glob

In [132]:
from sklearn.metrics import top_k_accuracy_score


def evaluate_model(model, pl_trainer, inf_loader, selected_metrics=None, topk=5):
    model.eval()
    metrics, times, inf_test_embeddings = compute_metrics(model, pl_trainer, inf_loader, selected_metrics)
    targets_df = targets.set_index("customer_id")
    inf_test_df = inf_test_embeddings.merge(targets_df, how="inner", on="customer_id").set_index("customer_id")
    
    X = inf_test_df.drop(columns=["gender"])
    y = inf_test_df["gender"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    classifier = catboost.CatBoostClassifier(
        iterations=150,
        random_seed=42,
        verbose=0,
    )
    classifier.fit(X_train, y_train)
    
    accuracy = classifier.score(X_test, y_test)
    
    return metrics, times, accuracy

In [131]:
def compute_metrics(model, pl_trainer, inf_loader, selected_metrics=None):
    """
    Compute selected embedding quality metrics.

    Parameters:
        model (torch.nn.Module): Trained model to evaluate.
        pl_trainer (pl.Trainer): PyTorch Lightning trainer for inference.
        inf_test_loader (DataLoader): DataLoader for inference.
        selected_metrics (list, optional): List of metric names to compute. 
                                           If None, computes all available metrics.

    Returns:
        dict: Computed metrics.
        dict: Computed times for each metric.
        pd.DataFrame: Extracted embeddings.
    """
    model.eval()
    
    inference_module = InferenceModuleMultimodal(
        model=model,
        pandas_output=True,
        drop_seq_features=True,
        model_out_name="emb",
        col_id="customer_id",
    )
    inference_module.model.is_reduce_sequence = True
    device = next(model.parameters()).device

    # for i, batch in enumerate(inf_train_loader):
    #     payload = batch[0].payload
    #     for key, value in payload.items():
    #         if isinstance(value, torch.Tensor) and value.numel() == 0:
    #             print(f"Empty tensor found in batch {i}, key={key}")

    
    # Получение эмбеддингов
    inf_embeddings = pd.concat(
        pl_trainer.predict(inference_module, inf_loader),
        axis=0,
    )
    embeddings = torch.tensor(inf_embeddings.drop(columns=["customer_id"]).values, device=device)
    embeddings_np = embeddings.cpu().numpy().astype(np.float32)

    # Предварительный расчет SVD
    u, s, _ = np.linalg.svd(embeddings_np, compute_uv=True, full_matrices=False)

    # Доступные метрики
    available_metrics = {
        "rankme": rankme,
        "coherence": coherence,
        "pseudo_condition_number": pseudo_condition_number,
        "alpha_req": alpha_req,
        "stable_rank": stable_rank,
        "ne_sum": ne_sum,
        "self_clustering": self_clustering,
        "ripser": ripser_metric
    }

    # Если selected_metrics не указаны, вычисляем все доступные метрики
    if selected_metrics is None:
        selected_metrics = list(available_metrics.keys())

    metrics = {}
    times = {}

    # Подсчет метрик и времени
    for metric_name in selected_metrics:
        if metric_name in available_metrics:
            if metric_name == "ripser":
                metrics[metric_name], times[metric_name] = available_metrics[metric_name](embeddings_np)
            else:
                start_time = time()
                metrics[metric_name] = available_metrics[metric_name](embeddings_np, u=u, s=s)
                times[metric_name] = time() - start_time

            print(f"Computed {metric_name} in {times[metric_name]:.4f} seconds")
        else:
            print(f"metric {metric_name} not is in available metrics. Choose one of these: {available_metrics.keys()}")

    return metrics, times, inf_embeddings

In [ ]:
# fixed_params = {
#     "batch_size": 64,
#     "learning_rate": 0.001,
#     "split_count": 3,
#     "cnt_min": 10,
#     "cnt_max": 50,
#     "embedding_dim": 16,  # Размерность эмбеддингов
#     "category_embedding_dim": 8,  # Размерность категорий эмбеддингов
#     "hidden_size": 128,  # Размер скрытого слоя по умолчанию
# }

# # Список гиперпараметров для перебора
# variable_params = {
#     "batch_size": [16, 32, 64, 128], 
#     "learning_rate": [0.0001, 0.001, 0.005, 0.01, 0.05],
#     "split_count": [2, 3, 5, 7],
#     "cnt_min": [5, 10, 15, 20, 25],
#     "cnt_max": [50, 80, 100, 150],
#     "embedding_dim": [8, 16, 24, 32],
#     "category_embedding_dim": [4, 8, 12, 16, 24],
#     "hidden_size": [32, 64, 128, 192, 256, 1024],
# }

# # Создание списка всех гиперпараметров, которые нужно перебрать
# all_hyperparameter_grids = []
# for variable_param_name, variable_param_values in variable_params.items():
#     for value in variable_param_values:
#         hyperparameter_grid = {**fixed_params, variable_param_name: value}
#         all_hyperparameter_grids.append((variable_param_name, hyperparameter_grid))


In [118]:
fixed_params = {
    "batch_size": 64,
    "learning_rate": 0.001,
    "split_count": 3,
    "cnt_min": 10,
    "cnt_max": 50,
    "embedding_dim": 16,  
    "category_embedding_dim": 8,  
}

# Перебираемые значения hidden_size
hidden_sizes = [64, 256, 512, 1024, 1696, 2048, 2424]  # От маленьких значений до очень больших

# Формирование гиперпараметров для перебора
all_hyperparameter_grids = [
    {**fixed_params, "hidden_size": h_size} for h_size in hidden_sizes
]

In [119]:
num_epochs = 30
output_csv = "hidden_size_results.csv"
columns = [
    *fixed_params.keys(), "checkpoint", "epoch_num", "accuracy", "early_stop_epoch", "hidden_size",
] + [
    "metric_" + key for key in [
        "rankme", "coherence", "pseudo_condition_number", 
        "alpha_req", "stable_rank", "ne_sum", "self_clustering", "ripser"
    ]
] + ["time_" + key for key in [
    "rankme", "coherence", "pseudo_condition_number", 
    "alpha_req", "stable_rank", "ne_sum", "self_clustering", "ripser"
]]

In [120]:
class CustomLogger(pl.Callback):
    def __init__(self):
        super().__init__()
        self.early_stopping_epoch = None  # Запомним, на какой эпохе произошла остановка
    
    def on_train_epoch_end(self, trainer, pl_module):
        train_loss = trainer.callback_metrics.get("train_loss", None)
        val_loss = trainer.callback_metrics.get("val_loss", None)
        
        if train_loss is not None and val_loss is not None:
            print(f"Epoch {trainer.current_epoch}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        
        # Если валидационный лосс увеличивается - фиксируем эпоху остановки
        if trainer.early_stopping_callback is not None and trainer.early_stopping_callback.wait_count == 0:
            self.early_stopping_epoch = trainer.current_epoch


custom_logger = CustomLogger()
early_stopping_callback = EarlyStopping(
    monitor="val_loss",
    patience=5,
    mode="min",
    verbose=True
)

In [121]:
import gc

In [122]:
# ! rm -rf checkpoints

In [123]:
# cur_time = time()

# for param in all_hyperparameter_grids:
    
#     # print(f'All params are frozen except {param[0]}')
#     params = param
#     # print(f"Testing parameters: {params}")

#     train_loader = create_datasets(train_dict, valid_dict, params, source_features)

#     sourceA_encoder_params = dict(
#         embeddings_noise=0.003,
#         linear_projection_size=64,
#         embeddings={
#             "mcc_code": {"in": mcc_code_in, "out": 32},
#             "term_id": {"in": term_id_in, "out": 32},
#         },
#     )
    
#     sourceB_encoder_params = dict(
#         embeddings_noise=0.003,
#         linear_projection_size=64,
#         embeddings={
#             "tr_type": {"in": tr_type_in, "out": 32},
#         },
#         numeric_values={"amount": "identity"},
#     )
    
#     sourceA_encoder = TrxEncoder(**sourceA_encoder_params)
#     sourceB_encoder = TrxEncoder(**sourceB_encoder_params)
    
#     seq_encoder = MultiModalSortTimeSeqEncoderContainer(
#         trx_encoders={
#             "sourceA": sourceA_encoder,
#             "sourceB": sourceB_encoder,
#         },
#         input_size=64,
#         hidden_size=params["hidden_size"],  # Используем только текущее значение hidden_size
#         seq_encoder_cls=RnnEncoder,
#         type="gru",
#     )

#     model = CoLESModule(
#         seq_encoder=seq_encoder,
#         optimizer_partial=partial(torch.optim.Adam, lr=params["learning_rate"]),
#         lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=10, gamma=0.5),
#     )

#     early_stopping_callback = EarlyStopping(
#         monitor="loss",
#         patience=5,
#         mode="min",
#         verbose=True
#     )

#     checkpoint_callback = ModelCheckpoint(
#         dirpath=checkpoints_path,
#         filename=f"model_{params['batch_size']}_{params['learning_rate']}_{params['split_count']}_{params['cnt_min']}_{params['cnt_max']}_{params['hidden_size']}{{epoch:02d}}",
#         save_top_k=-1,
#         every_n_epochs=1,
#     )

#     # Обучение модели
#     pl_trainer = pl.Trainer(
#         callbacks=[checkpoint_callback, early_stopping_callback, custom_logger],
#         default_root_dir=checkpoints_path,
#         check_val_every_n_epoch=1,
#         max_epochs=num_epochs,
#         accelerator="gpu",
#         devices=1,
#         enable_progress_bar=True,
#         precision=16
#     )
#     model.train()
#     pl_trainer.fit(model, train_loader)

#     early_stop_epoch = custom_logger.early_stopping_epoch
#     if early_stop_epoch is None:
#         early_stop_epoch = num_epochs

#     # Обработка чекпоинтов
#     checkpoint_files = glob.glob(f"{checkpoints_path}/model_{params['batch_size']}_{params['learning_rate']}_{params['split_count']}_{params['cnt_min']}_{params['cnt_max']}_{params['hidden_size']}*.ckpt")
#     checkpoint_files.sort()
#     print(f"Elapsed time: {time() - cur_time:.2f} seconds")

#     print(f'Early stop is {early_stop_epoch}')

#     for i, checkpoint in enumerate(checkpoint_files):
#         print(f"Processing checkpoint number {i}")
#         model = CoLESModule.load_from_checkpoint(checkpoint, seq_encoder=seq_encoder)

#         # Вычисление метрик и времени
#         metrics, times, accuracy = evaluate_model(model, checkpoint)
#         metrics_flattened = {f"metric_{k}": round(v, 4) for k, v in metrics.items()}
#         times_flattened = {f"time_{k}": round(v, 4) for k, v in times.items()}

#         # Сохранение результатов
#         new_result = {
#             **params,
#             "checkpoint": checkpoint,
#             "epoch_num": int(i),
#             "accuracy": accuracy,
#             **metrics_flattened,
#             **times_flattened,
#             "early_stop_epoch": int(early_stop_epoch)
#         }

#         # Сохранение в CSV
#         results = pd.DataFrame([new_result], columns=columns)
#         print('----------')
#         print(results["early_stop_epoch"])

#         if not os.path.exists(output_csv):  
#             pd.DataFrame(columns=columns).to_csv(output_csv, mode="w", index=False, header=True)
        
#         results.to_csv(output_csv, mode="a", header=False, index=False)

#         del metrics, accuracy, new_result
#         torch.cuda.empty_cache()
#         gc.collect()

#     print(f"Removing checkpoints for parameters: {params}")
#     for checkpoint in checkpoint_files:
#         os.remove(checkpoint)

#     del model
#     del train_loader
#     torch.cuda.empty_cache()
#     gc.collect()

# print("Optimization complete!")

## Try Optuna and recall TopK

In [124]:
import optuna
import pandas as pd
import torch
import gc
from tqdm import tqdm

# Файл для сохранения результатов
output_csv = "optuna_hyperparameter_by_topK.csv"

# Определение диапазонов гиперпараметров
def define_search_space(trial):
    return {
        "batch_size": trial.suggest_categorical("batch_size", [32, 64]),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-4, 5e-2),
        "hidden_size": trial.suggest_categorical("hidden_size", [64, 128]),
        "embedding_dim": trial.suggest_categorical("embedding_dim", [8, 16, 32]),
        "category_embedding_dim": trial.suggest_categorical("category_embedding_dim", [4, 8, 16]),
        "split_count": trial.suggest_categorical("split_count", [2, 3, 5]),
        "cnt_min": trial.suggest_categorical("cnt_min", [5, 10, 20]),
        "cnt_max": trial.suggest_categorical("cnt_max", [50, 80, 100]),
    }

# Метрики, по которым проводится подбор гиперпараметров
metric_names = [
    "rankme", "coherence", "pseudo_condition_number",
    "alpha_req", "stable_rank", "ne_sum", "self_clustering", "ripser"
]

# Список для хранения всех результатов
optuna_results = []

In [125]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [126]:
# from ptls.data_load.padded_batch import PaddedBatch


# class CustomCoLESModule(CoLESModule):
#     def __init__(self, custom_metric_name, *args, **kwargs):
#         super().__init__(*args, **kwargs)
#         self.custom_metric_name = custom_metric_name
#         # model.to(device)


#     def validation_step(self, batch, batch_idx):
#         print("valedation step")
#         x, y = batch
#         y = y.to(self.device)
#         for key in x:
#             if isinstance(x[key], PaddedBatch):
#                 # 🔄 Создаем новый PaddedBatch с seq_lens и payload на нужном устройстве
#                 x[key] = PaddedBatch(
#                     payload={k: v.to(self.device) for k, v in x[key].payload.items()},
#                     length=x[key].length.to(self.device)  # ⚠️ Используем length вместо seq_lens
#                 )
#             else:
#                 print(f"⚠️ [WARNING] Expected PaddedBatch but got {type(x[key])} for {key}")

#         print(f"Model is on device: {next(self.parameters()).device}")
#         print(f"x is on device: {[x[k].device for k in x]}")
#         print(f"y is on device: {y.device}")

#         y_hat = self(x)

#         # Compute loss (assuming classification task)
#         loss = torch.nn.functional.cross_entropy(y_hat, y)

#         metric_value, _, _ = compute_metrics(model, pl_trainer, inf_test_loader, selected_metrics=[self.custom_metric_name])

#         print(f"[DEBUG] Logging metric: valid/{self.custom_metric_name} = {metric_value[self.custom_metric_name]}")

#         self.trainer.logger.log_metrics({f"valid/{self.custom_metric_name}": metric_value[self.custom_metric_name]}, step=self.current_epoch)

#         return {"loss": loss, self.custom_metric_name: metric_value[self.custom_metric_name]}

In [135]:
! rm -rf /kaggle/working/checkpoints

In [136]:
! rm /kaggle/working/optuna_best_trials_accuracy.csv

In [133]:
import optuna
import time
import pandas as pd
import torch
import gc
import os
import glob
from functools import partial
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from tqdm import tqdm
from time import time

# Файл для логирования результатов
output_csv = "optuna_results.csv"

# Метрики для проверки
metric_names = [
    "rankme", "coherence", "pseudo_condition_number",
    "alpha_req", "stable_rank", "ne_sum", "self_clustering", "ripser"
]

optuna_columns = [
    *fixed_params.keys(), "checkpoint", "epoch_num", "accuracy", "topk_accuracy", "early_stop_epoch", "hidden_size",
] + [
    "metric_" + key for key in [
        "rankme", "coherence", "pseudo_condition_number", 
        "alpha_req", "stable_rank", "ne_sum", "self_clustering", "ripser"
    ]
] + ["time_" + key for key in [
    "rankme", "coherence", "pseudo_condition_number", 
    "alpha_req", "stable_rank", "ne_sum", "self_clustering", "ripser"
]]


def objective(trial):
    # print(f'dealing with metric {metric_name}')
    torch.cuda.empty_cache()
    gc.collect()

    params = define_search_space(trial)

    # === Dataset ===
    data_module = create_datasets(train_dict, valid_dict, params, source_features)

    # === Encoders ===
    sourceA_encoder = TrxEncoder(
        embeddings_noise=0.003,
        linear_projection_size=64,
        embeddings={
            "mcc_code": {"in": mcc_code_in, "out": 32},
            "term_id": {"in": term_id_in, "out": 32},
        },
    )

    sourceB_encoder = TrxEncoder(
        embeddings_noise=0.003,
        linear_projection_size=64,
        embeddings={
            "tr_type": {"in": tr_type_in, "out": 32},
        },
        numeric_values={"amount": "identity"},
    )

    seq_encoder = MultiModalSortTimeSeqEncoderContainer(
        trx_encoders={"sourceA": sourceA_encoder, "sourceB": sourceB_encoder},
        input_size=64,
        hidden_size=params["hidden_size"],
        seq_encoder_cls=RnnEncoder,
        type="gru",
    )

    model = CoLESModule(
        seq_encoder=seq_encoder,
        optimizer_partial=partial(torch.optim.Adam, lr=params["learning_rate"]),
        lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=10, gamma=0.5),
    )

    # === Callbacks ===
    early_stopping_callback = EarlyStopping(
        monitor=f"valid/recall_top_k", patience=5, mode="max", verbose=True
    )

    checkpoint_callback = ModelCheckpoint(
        dirpath=checkpoints_path,
        filename=f"model_optuna_trial_{trial.number}_epoch={{epoch:02d}}",
        save_top_k=-1,
        monitor="valid/recall_top_k",
        mode="max",
    )

    trainer = pl.Trainer(
        callbacks=[checkpoint_callback, early_stopping_callback, custom_logger],
        default_root_dir=checkpoints_path,
        check_val_every_n_epoch=1,
        max_epochs= 1, # num_epochs,
        accelerator="gpu",
        devices=1,
        enable_progress_bar=True,
        precision=16,
    )

    trainer.fit(model, datamodule=data_module)

    early_stop_epoch = custom_logger.early_stopping_epoch or num_epochs

    # === Evaluate checkpoints ===
    checkpoint_files = sorted(
        glob.glob(f"{checkpoints_path}/model_optuna_trial_{trial.number}_epoch=*.ckpt")
    )

    best_acc = float("-inf")

    for i, checkpoint in enumerate(checkpoint_files):
        model = CoLESModule.load_from_checkpoint(checkpoint, seq_encoder=seq_encoder)
        metrics, times, acc = evaluate_model(model, trainer, inf_train_loader)

        metrics_flattened = {f"metric_{k}": round(v, 4) for k, v in metrics.items()}
        times_flattened = {f"time_{k}": round(v, 4) for k, v in times.items()}
        trainer = pl.Trainer(accelerator="gpu", devices=1)

        val_metrics = trainer.validate(model=model, datamodule=data_module)
        recall_top_k = val_metrics[0].get("valid/recall_top_k", None)
        result = {
            **params,
            "checkpoint": checkpoint,
            "epoch_num": i,
            "accuracy": acc,
            "topk_accuracy": recall_top_k,
            **metrics_flattened,
            **times_flattened,
            "early_stop_epoch": early_stop_epoch,
        }

        results_df = pd.DataFrame([result], columns=optuna_columns)

        if not os.path.exists(output_csv):
            pd.DataFrame(columns=optuna_columns).to_csv(output_csv, index=False, header=True)
        results_df.to_csv(output_csv, mode="a", header=False, index=False)

        # current_metric_value = metrics.get(metric_name, float("-inf"))
        best_acc = max(best_acc, acc)

        del model, result, metrics
        torch.cuda.empty_cache()
        gc.collect()

    for ckpt in checkpoint_files:
        os.remove(ckpt)

    return best_acc


In [134]:
import optuna
import pandas as pd
import os
from time import time
from functools import partial

# Параметры для Optuna
num_trials = 10
cur_time = time()

# Путь к файлу для сохранения лучших результатов
best_trials_csv = "optuna_best_trials_accuracy.csv"

# Если файла ещё нет, создаём его
if not os.path.exists(best_trials_csv):
    pd.DataFrame(columns=["value", *fixed_params.keys()]).to_csv(best_trials_csv, index=False)

# Теперь оптимизируем только по accuracy
study = optuna.create_study(direction="maximize")  # Оптимизируем именно accuracy!
study.optimize(objective, n_trials=num_trials)     # objective теперь должна возвращать 1-accuracy или -accuracy

# Достаем лучший результат
best_trial = study.best_trial
best_result = {
    "value": best_trial.value,
    **best_trial.params
}

# Сохраняем лучший результат
df_best = pd.DataFrame([best_result])
df_best.to_csv(best_trials_csv, mode="a", header=False, index=False)

# Логи
print(f"✅ Optimization completed (direction: maximize)")
print(f"⏱️ Time passed: {time() - cur_time:.2f} sec")
print(f"🥇 Best trial value: {best_trial.value}")
print(f"📊 Params: {best_trial.params}")


[I 2025-05-02 09:55:09,194] A new study created in memory with name: no-name-f248933d-70a1-466f-9619-faa4bc0b2d23


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

[W 2025-05-02 09:55:44,085] Trial 0 failed with parameters: {'batch_size': 64, 'learning_rate': 0.03662649230096382, 'hidden_size': 128, 'embedding_dim': 16, 'category_embedding_dim': 4, 'split_count': 3, 'cnt_min': 20, 'cnt_max': 50} because of the following error: MisconfigurationException('`predict_dataloader` must be implemented to be used with the Lightning Trainer').
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-133-ffa6999e6f0c>", line 116, in objective
    metrics, times, acc = evaluate_model(model, trainer, train_loader)
  File "<ipython-input-132-c3b6016b4b3a>", line 6, in evaluate_model
    metrics, times, inf_test_embeddings = compute_metrics(model, pl_trainer, inf_loader, selected_metrics)
  File "<ipython-input-131-e8eb93936913>", line 38, in compute_metrics
    pl_trainer.predict(inference_module, inf_loader),
  File "/usr/loca

MisconfigurationException: `predict_dataloader` must be implemented to be used with the Lightning Trainer

## Eval model with best hyperparams

In [ ]:
input_csv = "/kaggle/input/gender-tr-best-params/gender_tr_optuna_best_params.csv"
best_trials_df = pd.read_csv(input_csv)

In [ ]:
best_trials_df.reset_index(inplace=True)
best_trials_df.rename(columns={"index": "metric", "metric":"value",
                              "value":"batch_size", "batch_size":"learning_rate", "learning_rate":"hidden_size"}, inplace=True)

In [ ]:
best_trials_df.rename(columns={"cnt_min": "embedding_dim", "embedding_dim":"cnt_min",
                              "category_embedding_dim":"cnt_max", "cnt_max":"category_embedding_dim"}, inplace=True)

In [ ]:
best_trials_df

In [ ]:
! rm -rf /kaggle/working/checkpoints

In [ ]:
! rm /kaggle/working/optuna_best_metrics_eval.csv

In [ ]:
# import pandas as pd
# import torch
# import gc
# import os
# import glob
# from functools import partial
# from time import time
# from pytorch_lightning import Trainer
# from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
# from itertools import islice



# checkpoints_path = "checkpoints"
# os.makedirs(checkpoints_path, exist_ok=True)


# # Для хранения результатов
# columns = list(best_trials_df.columns) + [
#     "checkpoint", "epoch_num", "accuracy", "early_stop_epoch"
# ] + [f"metric_{m}" for m in metric_names] + [f"time_{m}" for m in metric_names]

# for idx, row in islice(best_trials_df.iterrows(), 2, None):
#     metric_name = row["metric"]
#     print(f"\n=== Processing best params for metric: {metric_name} ===")
#     output_csv = f"optuna_best_metrics_eval_{metric_name}.csv"

#     print(row)
    
#     # Собираем параметры
#     params = {
#         "batch_size": int(row["batch_size"]),
#         "learning_rate": float(row["learning_rate"]),
#         "split_count": int(row["split_count"]),
#         "cnt_min": int(row["cnt_min"]),
#         "cnt_max": int(row["cnt_max"]),
#         "embedding_dim": int(row["embedding_dim"]),
#         "category_embedding_dim": int(row["category_embedding_dim"]),
#         "hidden_size": int(row["hidden_size"]),  # добавь сюда если будет в CSV
#     }

#     # Загружаем датасет
#     train_loader = create_datasets(train_dict, valid_dict, params, source_features)

#     # Создаём энкодеры
#     sourceA_encoder = TrxEncoder(
#         embeddings={"mcc_code": {"in": mcc_code_in, "out": 32}, "term_id": {"in": term_id_in, "out": 32}},
#         embeddings_noise=0.003,
#         linear_projection_size=64,
#     )
#     sourceB_encoder = TrxEncoder(
#         embeddings={"tr_type": {"in": tr_type_in, "out": 32}},
#         numeric_values={"amount": "identity"},
#         embeddings_noise=0.003,
#         linear_projection_size=64,
#     )

#     seq_encoder = MultiModalSortTimeSeqEncoderContainer(
#         trx_encoders={"sourceA": sourceA_encoder, "sourceB": sourceB_encoder},
#         input_size=64,
#         hidden_size=params["hidden_size"],
#         seq_encoder_cls=RnnEncoder,
#         type="gru",
#     )

#     model = CoLESModule(
#         seq_encoder=seq_encoder,
#         optimizer_partial=partial(torch.optim.Adam, lr=params["learning_rate"]),
#         lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=10, gamma=0.5),
#     )

#     early_stopping_callback = EarlyStopping(
#         monitor="loss", patience=5, mode="min", verbose=True
#     )
#     checkpoint_callback = ModelCheckpoint(
#         dirpath=checkpoints_path,
#         filename=f"best_{metric_name}_trial_{idx}_epoch={{epoch:02d}}",
#         save_top_k=-1,
#         every_n_epochs=1,
#     )

#     trainer = Trainer(
#         callbacks=[checkpoint_callback, early_stopping_callback],
#         default_root_dir=checkpoints_path,
#         check_val_every_n_epoch=1,
#         max_epochs=30,
#         accelerator="gpu",
#         devices=1,
#         enable_progress_bar=True,
#         precision=16
#     )

#     # Обучение
#     trainer.fit(model, train_loader)
#     early_stop_epoch = getattr(trainer.logger, "early_stopping_epoch", None) or num_epochs

#     # Вычисление метрик
#     checkpoint_files = sorted(
#         glob.glob(f"{checkpoints_path}/best_{metric_name}_trial_{idx}_epoch=*.ckpt")
#     )
#     model.cpu()
#     del model
#     torch.cuda.empty_cache()

#     for i, checkpoint in enumerate(checkpoint_files):
#         print(f"Evaluating checkpoint #{i}")
#         model = CoLESModule.load_from_checkpoint(checkpoint, seq_encoder=seq_encoder)
#         metrics, times, accuracy = evaluate_model(model, trainer)

#         row_result = {
#             **params,
#             "metric": metric_name,
#             "checkpoint": checkpoint,
#             "epoch_num": i,
#             "accuracy": accuracy,
#             "early_stop_epoch": early_stop_epoch,
#             **{f"metric_{k}": round(v, 4) for k, v in metrics.items()},
#             **{f"time_{k}": round(v, 4) for k, v in times.items()}
#         }

#         # Сохраняем результат
#         result_df = pd.DataFrame([row_result], columns=columns)
#         if not os.path.exists(output_csv):
#             pd.DataFrame(columns=columns).to_csv(output_csv, index=False)
#         result_df.to_csv(output_csv, mode="a", index=False, header=False)

#         del model, result_df
#         torch.cuda.empty_cache()
#         gc.collect()

#     # Удаление чекпоинтов
#     for ckpt in checkpoint_files:
#         os.remove(ckpt)
#     del trainer, train_loader, seq_encoder
#     torch.cuda.empty_cache()
#     gc.collect()

# print("✅ Evaluation of best params complete.")

In [ ]:
# import os

# # Загружаем сохраненные результаты, если файл уже существует
# if os.path.exists(output_csv):
#     print('exists')
#     processed_df = pd.read_csv(output_csv)
    
#     processed_params = [
#     tuple(row[["batch_size", "learning_rate", "split_count", "cnt_min", "cnt_max", "embedding_dim", "category_embedding_dim", "hidden_size"]])
#     for _, row in processed_df.iterrows()
# ]

# else:
#     processed_params = set()
#     print("don't exists")

In [ ]:
# remaining_hyperparameter_grids = []

# for variable_param_name, hyperparameter_grid in all_hyperparameter_grids:
#     # Преобразуем hyperparameter_grid в кортеж значений (только значимые параметры)
#     param_values_tuple = tuple(hyperparameter_grid.values())  # Без сортировки!
#     if param_values_tuple not in processed_params:
#         print(param_values_tuple)
#         remaining_hyperparameter_grids.append((variable_param_name, hyperparameter_grid))
#     else:
#         print(1)

# print(f"Remaining hyperparameter sets to process: {len(remaining_hyperparameter_grids)}")


In [ ]:
# # Запускаем обучение только для оставшихся гиперпараметров
# for variable_param_name, params in remaining_hyperparameter_grids:
#     print(f'variable param is {variable_param_name}')
#     print(f"Processing parameters: {params}")

#     train_loader = create_datasets(train_dict, valid_dict, params, source_features)

#     sourceA_encoder_params = dict(
#         embeddings_noise=0.003,
#         linear_projection_size=64,
#         embeddings={
#             "mcc_code": {"in": mcc_code_in, "out": 32},
#             "term_id": {"in": term_id_in, "out": 32},
#         },
#     )

#     sourceB_encoder_params = dict(
#         embeddings_noise=0.003,
#         linear_projection_size=64,
#         embeddings={
#             "tr_type": {"in": tr_type_in, "out": 32},
#         },
#         numeric_values={"amount": "identity"},
#     )

#     sourceA_encoder = TrxEncoder(**sourceA_encoder_params)
#     sourceB_encoder = TrxEncoder(**sourceB_encoder_params)

#     seq_encoder = MultiModalSortTimeSeqEncoderContainer(
#         trx_encoders={
#             "sourceA": sourceA_encoder,
#             "sourceB": sourceB_encoder,
#         },
#         input_size=64,
#         hidden_size=params["hidden_size"],
#         seq_encoder_cls=RnnEncoder,
#         type="gru",
#     )

#     model = CoLESModule(
#         seq_encoder=seq_encoder,
#         optimizer_partial=partial(torch.optim.Adam, lr=params["learning_rate"]),
#         lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=10, gamma=0.5),
#     )

#     early_stopping_callback = EarlyStopping(
#         monitor="loss",  # Следим за валидационным лоссом
#         patience=5,  # Количество эпох без улучшения перед остановкой
#         mode="min",  # Нужно минимизировать loss
#         verbose=True
#     )

#     checkpoint_callback = ModelCheckpoint(
#         dirpath=checkpoints_path,
#         filename=f"model_{params['batch_size']}_{params['learning_rate']}_{params['split_count']}_{params['cnt_min']}_{params['cnt_max']}_{params['hidden_size']}{{epoch:02d}}",
#         save_top_k=-1,
#         every_n_epochs=1,
#     )

#     # Обучение модели
#     pl_trainer = pl.Trainer(
#         callbacks=[checkpoint_callback, early_stopping_callback, custom_logger],
#         default_root_dir=checkpoints_path,
#         check_val_every_n_epoch=1,
#         max_epochs=num_epochs,
#         accelerator="gpu",
#         devices=1,
#         enable_progress_bar=True,
#         precision=16
#     )
#     model.train()
#     pl_trainer.fit(model, train_loader)

#     early_stop_epoch = custom_logger.early_stopping_epoch
#     if early_stop_epoch is None:
#         early_stop_epoch = num_epochs

#     # Обработка чекпоинтов
#     checkpoint_files = glob.glob(f"{checkpoints_path}/model_{params['batch_size']}_{params['learning_rate']}_{params['split_count']}_{params['cnt_min']}_{params['cnt_max']}_{params['hidden_size']}*.ckpt")
#     checkpoint_files.sort()
#     print(f"Elapsed time: {time() - cur_time:.2f} seconds")

#     print(f'Early stop is {early_stop_epoch}')

#     for i, checkpoint in enumerate(checkpoint_files):
#         print(f"Processing checkpoint number {i}")
#         model = CoLESModule.load_from_checkpoint(checkpoint, seq_encoder=seq_encoder)

#         metrics, times, accuracy = evaluate_model(model)
#         metrics_flattened = {f"metric_{k}": round(v, 4) for k, v in metrics.items()}
#         times_flattened = {f"time_{k}": round(v, 4) for k, v in times.items()}

#         new_result = {
#             **params,
#             "checkpoint": checkpoint,
#             "epoch_num": int(i),
#             "accuracy": accuracy,
#             **metrics_flattened,
#             **times_flattened,
#             "early_stop_epoch": int(early_stop_epoch)
#         }

#         new_result["epoch_num"] = int(new_result["epoch_num"])

#         results = pd.DataFrame([new_result], columns=columns)
#         print('----------')
#         print(results)

#         if not os.path.exists(output_csv):  # Проверяем, существует ли уже файл
#             pd.DataFrame(columns=columns).to_csv(output_csv, mode="w", index=False, header=True)
        
#         results.to_csv(output_csv, mode="a", header=False, index=False)

#         del metrics, accuracy, new_result
#         torch.cuda.empty_cache()
#         gc.collect()

#     print(f"Removing checkpoints for parameters: {params}")
#     for checkpoint in checkpoint_files:
#         os.remove(checkpoint)

#     del model
#     del train_loader
#     torch.cuda.empty_cache()
#     gc.collect()

# print("Optimization complete!")

In [ ]:
# df_results = pd.DataFrame(results)
# df_results.to_csv("CoLES_hyperparameter_results.csv", index=False)
# print("Results saved to hyperparameter_results.csv")

### Calc correlation

In [ ]:
hyperparameters = ["batch_size", "learning_rate", "split_count", "cnt_min", "cnt_max"]

# Корреляция между RankMe и accuracy
rankme_accuracy_corr, rankme_accuracy_pval = stats.pearsonr(df_results["rankme"], df_results["accuracy"])
print(f"Correlation between RankMe and Accuracy: {rankme_accuracy_corr:.4f}")
print(f"P-value: {rankme_accuracy_pval:.4e}\n")

# Корреляция между RankMe и каждым гиперпараметром
for param in hyperparameters:
    corr, pval = stats.pearsonr(df_results["rankme"], df_results[param])
    print(f"Correlation between RankMe and {param}: {corr:.4f}")
    print(f"P-value: {pval:.4e}\n")

### Plot smt

In [ ]:
hyperparameters = ["batch_size", "learning_rate", "split_count", "cnt_min", "cnt_max"]

# Построение графиков для accuracy
for param in hyperparameters:
    plt.figure(figsize=(8, 5))
    plt.scatter(df_results[param], df_results["accuracy"], alpha=0.7, label="Accuracy")
    plt.xlabel(param)
    plt.ylabel("Accuracy")
    plt.title(f"Accuracy vs {param}")
    plt.grid(True)
    plt.legend()
    plt.show()

In [ ]:
for param in hyperparameters:
    plt.figure(figsize=(8, 5))
    plt.scatter(df_results[param], df_results["rankme"], alpha=0.7, label="RankMe", color="orange")
    plt.xlabel(param)
    plt.ylabel("RankMe")
    plt.title(f"RankMe vs {param}")
    plt.grid(True)
    plt.legend()
    plt.show()